In [ ]:
# https://stackoverflow.com/questions/5099193/using-beautifulsoup-to-find-all-elements-starting-with-a-given-letter 

# undetected chromedriver
# #!/usr/bin/python3

# import pandas as pd
# import undetected_chromedriver as uc
# from bs4 import BeautifulSoup
# import selenium
# from selenium.webdriver.chrome.options import Options

# import time
# import sys
# import os
# import numpy as np

# options = Options()
# # options.add_argument('--incognito')
# driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=options)

# https://www.zenrows.com/blog/undetected-chromedriver#how-to-use-undetected-chromedriver 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random

In [ ]:
# Nusirašyti tamstai reikia tris funkcijas: getColNamesList, alterTableAddColumn ir writeMany

In [2]:
# Susirenkame šaldytuvų nuorodas.

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)

nuorodos=[]

for i in range(1, 34):
    url = f'https://www.varle.lt/saldytuvai/?p={i}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    varle = bs.find_all('div', {'class':'GRID_ITEM'})

    for saldytuvas in varle:
        try:
            nuorodos.append(saldytuvas.find('div', {'class':'product-title'}).find('a')['href'])
        except:
            pass

In [3]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 1'''.format(table=table)
    s = c.execute(sql)
    r = s.description
    return r

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(0.25)
            db.commit()
            # time.sleep(3)
    c.close()
    db.close()
    return True

def writeMany(data, table, dbname):
    connection = sqlite3.connect(dbname)
    cursor = connection.cursor()

    # Insert dictionary data into the table
    # columns = ', '.join(data.keys())  # Get column names as a string
    placeholders = ', '.join(['?'] * len(data))  # Generate placeholders for values
    values = tuple(data.values())  # Get values as a tuple

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {table} VALUES ({placeholders})"
    cursor.execute(sql, values)

    # Commit the changes and close the connection
    connection.commit()
    connection.close()
    return True


SDB = sqlite3.connect('varlesaldytuvai.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists saldytuvai
(
gamintojas text,
nuoroda text
)
 '''
Cs.execute(sql)



opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)

for nuor in range(0, 200):
    url = f'https://www.varle.lt{nuorodos[nuor]}' 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    specifikacija = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
    duom=specifikacija.find_all('li', {'class': 'spec-line'})

    stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
    duomenys = {key: None for key in stulpeliai}
    duomenys['nuoroda']= nuorodos[nuor]
    
    for i in duom:
        try:
            stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
            # print(i.find('span', {'class':'left-column'}).text.lower(), ' : ', i.find('span', {'class':'right-column'}).text.lower())
            kas = i.find('span', {'class':'left-column'}).text.lower()
            kaip = i.find('span', {'class':'right-column'}).text.lower()
            duomenys[kas]= kaip
        except:
            pass
    # print(duomenys)
    

    stulpeliai, *_ =zip(*getColNamesList('saldytuvai', 'varlesaldytuvai.db'))
    alterTableAddColumn(duomenys.keys(), stulpeliai, 'saldytuvai', 'varlesaldytuvai.db')
    writeMany(duomenys, 'saldytuvai', 'varlesaldytuvai.db')
    

In [4]:
print(duom)

[<li class="spec-line" itemprop="brand" itemscope="" itemtype="https://schema.org/Organization">
<span class="left-column">Gamintojas</span>
<span class="right-column" itemprop="name">Whirlpool</span>
</li>, <li class="spec-line">
<div class="left-column">Modelis</div>
<div class="right-column"><h2 itemprop="name">5511202</h2></div>
</li>, <li class="spec-line">
<span class="left-column">Kontrolė</span>
<span class="right-column" itemprop="name">Mechaninis</span>
</li>, <li class="spec-line">
<span class="left-column">Svoris</span>
<span class="right-column" itemprop="name">27</span>
</li>, <li class="spec-line">
<span class="left-column">Spalva</span>
<span class="right-column" itemprop="name">Balta</span>
</li>, <li class="spec-line">
<span class="left-column">Technologija</span>
<span class="right-column" itemprop="name">Šalčio nėra</span>
</li>, <li class="spec-line">
<span class="left-column">Triukšmo klasė</span>
<span class="right-column" itemprop="name">C</span>
</li>, <li clas

In [6]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)

for nuor in range(11, 15):
    url = f'https://www.varle.lt{nuorodos[nuor]}' 
    driver.get(url)
    time.sleep(3)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    specifikacija = bs.find('div', {'class' : 'accordion'}).find('div', {'class': 'PRODUCT_SPEC'})
    duom=specifikacija.find_all('li', {'class': 'spec-line'})

    duomenys = {}
    for i in duom:
        try:
            # print(i.find('span', {'class':'left-column'}).text.lower(), ' : ', i.find('span', {'class':'right-column'}).text.lower())
            duomenys[i.find('span', {'class':'left-column'}).text.lower()]= i.find('span', {'class':'right-column'}).text.lower()
        except:
            pass
    print(duomenys)

{'gamintojas': 'liebherr', 'svoris': '57.1', 'oro valymo filtras': 'yra', 'naudinga šaldytuvo talpa': '294', 'naujoji energijos vartojimo efektyvumo klasė': 'ir', 'apšvietimas': 'led', 'vėsaus stalčiaus teleskopiniai bėgeliai': 'yra', 'drėgmės išlaikymo technologija': 'yra', 'keičiama durų atidarymo kryptis': 'yra', 'atostogų funkcija': 'yra', 'gylis': '55.0', 'plotis': '56.0', 'aukštis': '177.2', 'minkštas durelių uždarymas': 'nėra', 'individualus lentynėlių išdėstymas durelėse': 'nėra', 'ventiliatorius': 'nėra', 'montavimo tipas': 'pastatytas', 'klimato klasė': 'sn-t', 'metalizuota galinė sienelė': 'nėra', 'greitas vėsinimas šaldytuve': 'yra', 'valdymas': 'internal touch control lcd', 'atvirų durų signalas': 'yra', 'spalva': 'balta', 'standartizuotas aukštis': '180', 'šaldymo technologijos tipas': 'statinis', 'ekonominis režimas': 'yra', 'naudinga talpa vėsaus stalčiaus': '98', 'vyrių tipas': 'slydimas', 'garso lygio klasė': 'c', 'tipas': 'fridge without freezer', 'vėsusis stalčius':